# 1. Import libraries to be used in this notebook

In [1]:
import numpy as np # for array manipulation and basic scientific calculation
import xarray as xr # To read NetCDF files
from scipy.interpolate import griddata # Simple regridding
from netCDF4 import Dataset # To write NetCDF files

# 2. Read a model output with unstructured grid (CONUS)

In [2]:
File_CONUS = '/glade/campaign/acom/acom-climate/tilmes/CO_CONUS/f.e22.FCcotagsNudged.ne0CONUSne30x8.cesm220.2012-01/atm/hist/f.e22.FCcotagsNudged.ne0CONUSne30x8.cesm220.2012-01.cam.h1.2013-08.nc'
ds_CONUS = xr.open_dataset( File_CONUS ) 


# 2. Regrid using linear and nearest interpolation 

In [3]:
# This will define the lat/lon range we are using to correspond to 0.1x0.1 over CONUS
lon2d = np.linspace(210,310,1001)
lat2d =  np.linspace(0,70,701)

# This will put lat and lon into arrays, something that is needed for plotting
X, Y = np.meshgrid(lon2d,lat2d)

# We will also need the model lat/lon values for regridding
mdllat = ds_CONUS['lat']
mdllon = ds_CONUS['lon']

# Here we use scipy to interpolate the 1D data to the prescribed 2D grid (nearest and linear)
o3near = griddata((mdllon,mdllat), ds_CONUS.isel(time=0,lev=31)['O3'], (X, Y), method='nearest')
o3lin = griddata((mdllon,mdllat), ds_CONUS.isel(time=0,lev=31)['O3'], (X, Y), method='linear')

# You can use mathematical functions to manipulate numpy arrays easily, i.e. for unit conversions
o3lin = o3lin*1e9

# 3. Write the output to a NetCDF file

In [4]:
# We have defined the file location here and opened the dataset
ds_output = Dataset('./CONUS_0.1x0.1.nc', 'w', format='NETCDF4')

# First create the dimension of the netcdf, for this example it is just lat and lon
lat = ds_output.createDimension('lat', np.size(lat2d))
lon = ds_output.createDimension('lon', np.size(lon2d))

# Then create the variables, this included the dimensions and tracers you want written into the file
lats = ds_output.createVariable('lat', 'f4', ('lat',))
lons = ds_output.createVariable('lon', 'f4', ('lon',))
O3near = ds_output.createVariable('O3near', 'f4', ('lat', 'lon',))
O3lin = ds_output.createVariable('O3lin', 'f4', ('lat', 'lon',))

# You can also assign attributes to the files as well, such as units or molecular weights
O3near.units = 'mol/mol'
O3lin.units = 'ppb'

# Now we can assign the values from the numpy arrays we have calculated above
lats[:] = lat2d
lons[:] = lon2d
O3near[:,:] = o3near
O3lin[:,:] = o3lin

# Be sure to close the netcdf at the end of each call or you may run into issues when trying to access it later in the code
ds_output.close()